# Run situational embedding benchmarks from Google Colab

## Clone benchmark repository locally
This use is typically thought for a use in a Google Colab environment.

In [ ]:
# Clone GitHub repository
import os

repo_url = "https://github.com/jacomyma/embedding-models-situational-benchmarks"
repo_name = repo_url.split('/')[-1]

if not os.path.exists(repo_name):
    !git clone {repo_url}
else:
    print(f"Repository '{repo_name}' already exists. Skipping clone.")

In [ ]:
# Move "benchmark" folder to the root
!mv {repo_name}/benchmark .

## Install dependencies

In [ ]:
# Run once
!pip install sentence-transformers datasets scipy scikit-learn pandas tqdm

## Setup & logging

In [ ]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s  %(levelname)-8s  %(message)s",
    datefmt="%H:%M:%S",
    stream=sys.stdout,
)

# Make sure the benchmark package is on the path
# (if running from the repo root this is not needed)
import os, sys
sys.path.insert(0, os.path.abspath(".."))

## Inspect available tasks

In [ ]:
from benchmark import TASK_REGISTRY

for name, task in TASK_REGISTRY.items():
    print(f"  {name:20s}  →  {task.description}")

## Define models to benchmark

Models are plain dicts — no code changes needed to add/remove models.

In [ ]:
MODEL_CONFIGS = [
    {
        "type": "sentence_transformer",
        "model": "sentence-transformers/all-MiniLM-L6-v2",
        # Fast, ~80 MB — good baseline
    },
    {
        "type": "sentence_transformer",
        "model": "BAAI/bge-small-en-v1.5",
        # Strong small model from BAAI
    },
    # Uncomment to add OpenAI (requires OPENAI_API_KEY):
    # {
    #     "type": "openai",
    #     "model": "text-embedding-3-small",
    # },
]

# Which tasks to run
TASK_NAMES = ["sts", "retrieval"]   # add "clustering" for a heavier run

## Run the benchmark

In [ ]:
from pathlib import Path
from benchmark import BenchmarkRunner

runner = BenchmarkRunner(
    model_configs=MODEL_CONFIGS,
    task_names=TASK_NAMES,
    output_dir=Path("results"),
    cache_dir=Path(".cache/embeddings"),
    batch_size=128,
    show_progress=True,
)

results = runner.run()

## Download results

In [ ]:
import zipfile
import os
from google.colab import files

output_dir = "results"
zip_file_name = f"{output_dir}.zip"

# Create a zip file of the results directory
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files_in_dir in os.walk(output_dir):
        for file in files_in_dir:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_dir))

print(f"Created '{zip_file_name}'.")

# Download the zip file
files.download(zip_file_name)